<table class="tfo-notebook-buttons" align="left">
  <td>
    <a target="_blank" href="https://colab.research.google.com/github/ENEE439d-TEXTML/TextML/blob/master/input.ipynb"><img src="https://www.tensorflow.org/images/colab_logo_32px.png" />Run in Google Colab</a>
  </td>
  <td>
    <a target="_blank" href="https://github.com/ENEE439d-TEXTML/TextML/blob/master/input.ipynb"><img src="https://www.tensorflow.org/images/GitHub-Mark-32px.png" />View source on GitHub</a>
  </td>
</table>

## Data input and ecoding testing

In [1]:
import pandas as pd
import numpy as np
from tqdm import tqdm
import tensorflow as tf
from tensorflow import keras
from keras import layers

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
filepath = '/content/drive/MyDrive/ml-data/data.csv' #'data.csv'
df= pd.read_csv(filepath)
df.head()

,no,paper,cited_paper,label,text
0,0,A00-1043,A00-2024,0,We analyzed a set of articles and identified s...
1,1,H05-1033,A00-2024,0,Table 3: Example compressions Compression AvgL...
2,2,I05-2009,A00-2024,0,5.3 Related works and discussion Our two-step ...
3,3,I05-2009,A00-2024,0,(1999) proposed a summarization system based o...
4,4,I05-2009,A00-2024,0,We found that the deletion of lead parts did n...


In [4]:
df.keys()

Index(['no', 'paper', 'cited_paper', 'label', 'text'], dtype='object')

In [5]:
df1=df.drop(['no','paper'],axis=1)
df1.head()

,cited_paper,label,text
0,A00-2024,0,We analyzed a set of articles and identified s...
1,A00-2024,0,Table 3: Example compressions Compression AvgL...
2,A00-2024,0,5.3 Related works and discussion Our two-step ...
3,A00-2024,0,(1999) proposed a summarization system based o...
4,A00-2024,0,We found that the deletion of lead parts did n...


In [49]:
check=df1['label']
values=df1.pivot_table(columns=['label'], aggfunc='size')
l=list(values)
print(values)
ratio=(l[0]+l[2])/l[1]
print("perect of non-neutral:", ratio)

label
-1     21
 0    898
 1     80
dtype: int64
perect of non-neutral: 0.11247216035634744


In [6]:
import pprint
pp= pprint.PrettyPrinter()

In [7]:
context=df1['text']
pp.pprint(context[0])
pp.pprint(context[1])
pp.pprint(context[2])
pp.pprint(context[3])
pp.pprint(context[4])

('We analyzed a set of articles and identified six major operations that can '
 'be used for editing the extracted sentences, including removing extraneous '
 'phrases from an extracted sentence, combining a reduced sentence with other '
 'sentences, syntactic transformation, substituting phrases in an extracted '
 'sentence with their paraphrases, substituting phrases with more general or '
 'specific descriptions, and reordering the extracted sentences (Jing and '
 'McKeown, 1999; Jing and McKeown, 2000).')
('Table 3: Example compressions Compression AvgLen Rating Baseline 9.70 1.93 '
 'BT-2-Step 22.06 3.21 Spade 19.09 3.10 Humans 20.07 3.83 Table 4: Mean '
 'ratings for automatic compressions nally, we added a simple baseline '
 'compression algorithm proposed by Jing and McKeown (2000) which removed all '
 'prepositional phrases, clauses, toinfinitives, and gerunds.')
('5.3 Related works and discussion Our two-step model essentially belongs to '
 'the same category as the works of 

In [8]:
import re

In [16]:
re1= "\(((([A-Za-z]+ *)+(, \d+))+(; )*)+\)" # matches author and author, year
re_year=",? \(?\d{4}\)?" # match , {4 digits} which may be wrapped in () 
re_et= "(\w+ *)et al\. ?"+re_year # matches author et al. , year
re_2a= "(\w+ *)and ((\w+ *))?"+re_year # matches author and author, year
re_sep="((; )|( and ))*"# match the '; ' gap or ' and ' gap
re_apa ="\(?("+"(\(?"+re_2a+"|"+re_et+"\)?)"+re_sep+")+"
p1= re.compile(re_apa)
a=p1.search(context[0])
print(a)
a=p1.search(context[2])
print(a)
#p1= re.compile(re_2a)
a=p1.search(context[4])
print(a)
print(a.groups())

<re.Match object; span=(437, 485), match='(Jing and McKeown, 1999; Jing and McKeown, 2000)'>
<re.Match object; span=(109, 158), match='(Mani et al. , 1999) and (Jing and McKeown, 2000)>
<re.Match object; span=(101, 124), match='Jing and McKeown (2000)'>
('Jing and McKeown (2000)', 'Jing and McKeown (2000)', 'Jing ', 'McKeown', 'McKeown', None, None, None, None)


In [18]:
def remove_citation(text,regex=re_apa):
  text1=text
  rem_len=0
  pattern= re.compile(regex)
  while True:
    matches=pattern.search(text1)
    # print(matches)
    if matches == None:
      break

    spn=matches.span()
    text1=text1[0:spn[0]]+text1[spn[1]:-1]
    cit_len=spn[1]-spn[0]
    rem_len+=cit_len
  
  percent_removed=rem_len/len(text)
  return text1,percent_removed 

print(context[5])
remove_citation(context[5],regex=re_apa)

Automatic text summarization approaches have offered reasonably well-performing approximations for identifiying important sentences (Lin and Hovy, 2002; Schiffman et al., 2002; Erkan and Radev, 2004; Mihalcea and Tarau, 2004; Daume III and Marcu, 2006) but, not surprisingly, text (re)generation has been a major challange despite some work on sub-sentential modification (Jing and McKeown, 2000; Knight and Marcu, 2000; Barzilay and McKeown, 2005).


('Automatic text summarization approaches have offered reasonably well-performing approximations for identifiying important sentences Daume  but, not surprisingly, text (re)generation has been a major challange despite some work on sub-sentential modification ',
 0.4209354120267261)

In [34]:
output=df1['text'].apply(lambda x: remove_citation(text=x)) #df['col1'] = df.apply(lambda x: complex_function(x['col1']), axis=1)
df_o = pd.DataFrame(list(output), columns =['clean','p_rem'])
df_o.head()

In [51]:
df1['text_clean']=df_o['clean']
df1['p_rem']=df_o['p_rem']
df1.head()

,cited_paper,label,text,text_clean,p_rem
0,A00-2024,0,We analyzed a set of articles and identified s...,We analyzed a set of articles and identified s...,0.098765
1,A00-2024,0,Table 3: Example compressions Compression AvgL...,Table 3: Example compressions Compression AvgL...,0.065903
2,A00-2024,0,5.3 Related works and discussion Our two-step ...,5.3 Related works and discussion Our two-step ...,0.308176
3,A00-2024,0,(1999) proposed a summarization system based o...,(1999) proposed a summarization system based o...,0.062500
4,A00-2024,0,We found that the deletion of lead parts did n...,We found that the deletion of lead parts did n...,0.184000
